In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset , DataLoader
import re
import random

In [3]:
# load & read file
dataset_name = "shakespeare.txt"
raw_data = open(dataset_name, "r").read().lower()

In [4]:
raw_data

'now,fair hippolyta,our nuptial hour draws on apace : four happy days bring in another moon;but o ! methinks how slow this old moon wanes;she lingers my desires,like to a step dame,or a dowager long withering out a young man\'s revenue.four days will quickly steep themselves in night;four nights will quickly dream away the time;and then the moon,like to a silver bow new-bent in heaven,shall behold the night of our solemnities. go,philostrate,stir up the athenian youth to merriments;awake the pert and nimble spirit of mirth;turn melancholy forth to funerals;the pale companion is not for our pomp.hippolyta,i woo\'d thee with my sword,and won thy love doing thee injuries;but i will wed thee in another key,with pomp,with triumph,and with revelling.happy be theseus,our renowned duke !thanks,good egeus : what\'s the news with thee ?full of vexation come i,with complaint against my child,my daughter hermia.stand forth,demetrius.my noble lord,this man hath my consent to marry her.stand forth,l

In [5]:
len(raw_data)

302259

In [6]:
#clean data
clean_data = re.sub(r'[^\w\s]', " ", raw_data)


In [7]:
clean_data

'now fair hippolyta our nuptial hour draws on apace   four happy days bring in another moon but o   methinks how slow this old moon wanes she lingers my desires like to a step dame or a dowager long withering out a young man s revenue four days will quickly steep themselves in night four nights will quickly dream away the time and then the moon like to a silver bow new bent in heaven shall behold the night of our solemnities  go philostrate stir up the athenian youth to merriments awake the pert and nimble spirit of mirth turn melancholy forth to funerals the pale companion is not for our pomp hippolyta i woo d thee with my sword and won thy love doing thee injuries but i will wed thee in another key with pomp with triumph and with revelling happy be theseus our renowned duke  thanks good egeus   what s the news with thee  full of vexation come i with complaint against my child my daughter hermia stand forth demetrius my noble lord this man hath my consent to marry her stand forth lysa

In [8]:
len(clean_data)

302259

# Creating Sequence

In [9]:
def create_seq(text,seq_len=8):
  
  
  sequence = []
  
  if len(text.split()) > seq_len:
    for i in range(seq_len,len(text.split())):
      #select the sequnece of tokens
      seq = text.split()[i-seq_len:i+1] 
      sequence.append(" ".join(seq))

    return sequence


  else:
    return [text]

In [10]:
raw_seq= create_seq(clean_data)

In [11]:
raw_seq

['now fair hippolyta our nuptial hour draws on apace',
 'fair hippolyta our nuptial hour draws on apace four',
 'hippolyta our nuptial hour draws on apace four happy',
 'our nuptial hour draws on apace four happy days',
 'nuptial hour draws on apace four happy days bring',
 'hour draws on apace four happy days bring in',
 'draws on apace four happy days bring in another',
 'on apace four happy days bring in another moon',
 'apace four happy days bring in another moon but',
 'four happy days bring in another moon but o',
 'happy days bring in another moon but o methinks',
 'days bring in another moon but o methinks how',
 'bring in another moon but o methinks how slow',
 'in another moon but o methinks how slow this',
 'another moon but o methinks how slow this old',
 'moon but o methinks how slow this old moon',
 'but o methinks how slow this old moon wanes',
 'o methinks how slow this old moon wanes she',
 'methinks how slow this old moon wanes she lingers',
 'how slow this old moon w

In [12]:
len(raw_seq)

60898

In [13]:
seq_new = []

for i in clean_data:
  for j in i:
    seq_new.append(j)
    

In [14]:
len(seq_new)

302259

In [15]:
#creat input & target sequence
x = []
y = []

for i in raw_seq:
  x.append(" ".join(i.split()[:-1]))
  y.append(" ".join(i.split()[1:]))

In [16]:
len(x) , len(y)

(60898, 60898)

In [17]:
# create int-to-word

int2word = {}
c = 0
uni_word = set((clean_data).split())

for index,word in enumerate(uni_word):
  int2word[index] = word

int_2_word = {index:word for index,word in enumerate(uni_word)}
word2int = {word:index for index,word in int2word.items()}

In [18]:
len(int_2_word)

6187

In [19]:
len(int2word)

6187

In [20]:
len(word2int)

6187

In [21]:
vocab_size = len(int2word)
vocab_size

6187

In [22]:
def get_int_seq(seq):
  return [word2int[w] for w in seq.split()]

In [23]:
#x and y covert in into int
x_int = [get_int_seq(i) for i in x]
y_int = [get_int_seq(j) for j in y]

In [24]:
#convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

In [25]:
x_int

array([[4699, 6048, 5395, ..., 4736, 1264, 1454],
       [6048, 5395, 2886, ..., 1264, 1454, 3816],
       [5395, 2886, 5208, ..., 1454, 3816,  458],
       ...,
       [5389, 4697, 3611, ..., 3830, 2578, 1929],
       [4697, 3611, 3912, ..., 2578, 1929, 1556],
       [3611, 3912, 4678, ..., 1929, 1556, 6048]])

In [26]:
len(x_int) , len(y_int)

(60898, 60898)

In [27]:
x_int.dtype

dtype('int64')

In [28]:
inputs = x_int
targets = y_int

In [29]:
#train , test & validation
train_ratio = 0.8
valid_ratio = (1-train_ratio)/2

total = len(x_int)
train_cutoff = int((total*train_ratio))
valid_cutoff = int((total*(1-valid_ratio)))


#numpy to tensor
train_x, train_y = torch.Tensor(inputs[:train_cutoff]).long() , torch.Tensor(targets[:train_cutoff]).long()
valid_x , valid_y = torch.Tensor(inputs[train_cutoff:valid_cutoff]).long() , torch.Tensor(targets[train_cutoff:valid_cutoff]).long()


train_data = TensorDataset(train_x,train_y)
valid_data = TensorDataset(valid_x ,valid_y)


batch_size = 32


train_loader = DataLoader(train_data,batch_size = batch_size,shuffle=True,drop_last=True)
valid_loader = DataLoader(valid_data,batch_size = batch_size,shuffle=True,drop_last=True)


In [30]:
for x , y in train_loader:
  print(x.shape,y.shape)
  break

torch.Size([32, 8]) torch.Size([32, 8])


In [31]:
x_int.shape

(60898, 8)

In [32]:
y_int.shape

(60898, 8)

In [33]:
class ShakespeareGenerationLSTM(nn.Module):

  def __init__(self,n_vocab,n_embed,n_hidden,n_layers,drop_pr = 0.25):
    super().__init__()
    self.n_vocab = n_vocab
    self.n_embed = n_embed
    self.n_hidden = n_hidden 
    self.drop_pr = drop_pr 
    self.n_layers = n_layers

    self.embedding = nn.Embedding(n_vocab,n_embed)
    self.lstm = nn.LSTM(n_embed,n_hidden,n_layers,batch_first = True)
    self.dropout = nn.Dropout(drop_pr)
    self.fc = nn.Linear(n_hidden,n_vocab)

  #(input , (hidden_state & cell state))
  def forward(self,x,hidden):
    
    #pass the input through embedding layers
    embedded = self.embedding(x)

    #Get the output and new hidden state and cell state
    n_out , hidden = self.lstm(embedded,hidden) #(lstm) (input,(hidden_state,cell_state)) 

    #pass the droupout
    out = self.dropout(n_out)

    #reshape
    #out = out.contingous().view(-1,self.n_hidden)
    out = out.reshape(-1,self.n_hidden)

    out = self.fc(out)

    return out,hidden


  def init_hidden(self,batch_size):
    weights = next(self.parameters()).data

    if (torch.cuda.is_available()):
      hidden = (weights.new(self.n_layers , batch_size,self.n_hidden).zero_().cuda(),
         weights.new(self.n_layers,batch_size,self.n_hidden).zero_().cuda())

    else:
      hidden = (weights.new(self.n_layers , batch_size,self.n_hidden).zero_(),
         weights.new(self.n_layers,batch_size,self.n_hidden).zero_())
      
    return hidden

In [34]:
#n_vocab,n_embed,n_hidden,n_layers,drop_pr = 0.25
n_vocab = len(int2word)
n_embed = 256
n_layers = 4
n_hidden = 128

model = ShakespeareGenerationLSTM(n_vocab,n_embed,n_hidden,n_layers,drop_pr = 0.25)

In [35]:
model
model.cuda()

ShakespeareGenerationLSTM(
  (embedding): Embedding(6187, 256)
  (lstm): LSTM(256, 128, num_layers=4, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=6187, bias=True)
)

In [36]:
print(model)

ShakespeareGenerationLSTM(
  (embedding): Embedding(6187, 256)
  (lstm): LSTM(256, 128, num_layers=4, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=6187, bias=True)
)


In [37]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [38]:
def train_sentiment(model,loss_fn,optimizer,epochs=20):

  for epoch in range(epochs):
    h = model.init_hidden(batch_size=32)
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0


    model.train()

    for input,labels in train_loader:
      h = tuple([i.data for i in h]) #tuple([hidden_state,cell_state])
      input , labels = input.cuda() , labels.cuda()
      #print(input.shape,labels.shape)
     
    
      output,h = model(input,h)
      optimizer.zero_grad()

      loss = loss_fn(output,labels.view(-1))
      loss.backward()
      train_loss += loss.item()

      optimizer.step()

    model.eval()

    for input,labels in valid_loader:
      input , labels = input.cuda() , labels.cuda()
      h = tuple([i.data for i in h]) #tuple([hidden_state,cell_state])
      output,h = model(input,h)
      

      loss = loss_fn(output,labels.view(-1))
      valid_loss += loss.item()

    print("Epoch {} , Train_loss : {:.2f},valid_loss : {:.2f}".format(
        epoch+1 , train_loss/len(train_loader),valid_loss/len(valid_loader)))

In [39]:
train_sentiment(model,loss_fn,optimizer,epochs=30)

Epoch 1 , Train_loss : 6.29,valid_loss : 6.58
Epoch 2 , Train_loss : 5.57,valid_loss : 6.81
Epoch 3 , Train_loss : 5.10,valid_loss : 7.14
Epoch 4 , Train_loss : 4.78,valid_loss : 7.38
Epoch 5 , Train_loss : 4.53,valid_loss : 7.59
Epoch 6 , Train_loss : 4.33,valid_loss : 7.82
Epoch 7 , Train_loss : 4.16,valid_loss : 8.07
Epoch 8 , Train_loss : 4.01,valid_loss : 8.28
Epoch 9 , Train_loss : 3.88,valid_loss : 8.48
Epoch 10 , Train_loss : 3.76,valid_loss : 8.68
Epoch 11 , Train_loss : 3.66,valid_loss : 8.79
Epoch 12 , Train_loss : 3.56,valid_loss : 9.00
Epoch 13 , Train_loss : 3.47,valid_loss : 9.17
Epoch 14 , Train_loss : 3.40,valid_loss : 9.31
Epoch 15 , Train_loss : 3.32,valid_loss : 9.43
Epoch 16 , Train_loss : 3.24,valid_loss : 9.55
Epoch 17 , Train_loss : 3.18,valid_loss : 9.63
Epoch 18 , Train_loss : 3.11,valid_loss : 9.73
Epoch 19 , Train_loss : 3.05,valid_loss : 9.87
Epoch 20 , Train_loss : 3.00,valid_loss : 9.94
Epoch 21 , Train_loss : 2.95,valid_loss : 10.01
Epoch 22 , Train_loss

In [40]:
def predict(model,tkn,h = None):
  x = np.array([[word2int[tkn]]])
  inputs = torch.from_numpy(x)
  inputs = inputs.cuda()
  h = tuple([i.data for i in h])

  # get the output of the model
  out , h = model(inputs,h)
  # get the token proba
  p = F.softmax(out,dim=1).data # cuda  

  p = p.cpu()
  p = p.numpy()

  p = p.reshape(p.shape[1],)

  #get indices of next or top 3 values
  top_n_indx = p.argsort()[-1:][::-1][0]

  #random selct one of the two indecies 

  return int2word[top_n_indx] , h

In [43]:
#function generate text
def generate_text(model,size,prime="she lingers my desires"):

  # To GPU
  model.cuda()

  model.eval()

  #batch size as 1
  h = model.init_hidden(1)

  tokens = prime.split() # ["it","is","good"]

  #predict next token
  for t in tokens:
    token , h = predict(model,t.lower(),h)

  tokens.append(token)

  #predict subsequent toekns

  for i in range(size):
    token , h  = predict(model,tokens[-1],h)
    tokens.append(token)

  return " ".join(tokens)

In [44]:
generate_text(model,100,"old moon wanes she lingers my desires")

'old moon wanes she lingers my desires like to a second fool captain if i come to the camp he does me wrong my lord if we must not so near at a man s daughter i am not my lord the youngest son of sweet music what my life is in the world that hatred is so far from jealousy to sleep by love s sight of thy misprision must perforce found for by his life s noontide with the antipodes it cannot be but i thank your majesty i will not bate thee a scruple well i shall be dogged with company and our devices known'